# Cosmic Neighbours

In this project we are adding a time dimension to the famous [Drake Equation](https://en.wikipedia.org/wiki/Drake_equation). The Drake Equation is as follows:
    
#### N = Rs * fp * ne * fl * fi * fc * L
    where:
    N = number of civilizations with which humans could communicate
    Rs = mean rate of star formation
    fp = fraction of stars that have planets
    ne = mean number of planets that could support life per star with planets
    fl = fraction of life-supporting planets that develop life
    fi = fraction of planets with life where life develops intelligence
    fc = fraction of intelligent civilizations that develop communication
    L = mean length of time that civilizations can communicate
    
This solution to the number of civilizations is a steady state solution. Meaning, we cannot use this estimate to estimate the number of extict civilizations in our galaxy. And it assumes that the intelligent population of the galaxy is not growing. I will try to answer these questions by adding a time dimension to the calcuation.


### Imports

In [1]:
from drakefunctions import *

import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:_.0f}'.format)
from scipy.stats import poisson
from scipy.special import gamma  # not the one from scipy.stats
from math import factorial, exp, log
import datetime, time

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib as mpl
%matplotlib inline
mpl.rcParams["axes.formatter.min_exponent"] = 20  # no scientific notation in graphs
plt.rcParams['axes.titley'] = 1.0    # y is in axes-relative coordinates.
plt.rcParams['axes.titlepad'] = 10   # pad is in points... default is 6
import seaborn as sns
sns.set_theme()

### The Original Drake Equation

In [ ]:
# Using our best estimates, the Drake Equation predicts...
RS = 2  # star formation per galaxy (ie, milky way)
FP = 1
NE = 0.4
FL = 1  # seems optimistic to me
FI = 1  # seems optimistic to me
FC = 0.1  # looks like a complete guess 
L = 1000  # maybe between 1000 and 100_000_000

Drake(RS, FP, NE, FL, FI, FC, L)

In [ ]:
# hack the Drake equation to estimate how many extict technological civilizations
# might exist in the Milky Way
print(f"{Drake(RS, FP, NE, FL, FI, FC, 15_000_000_000):,}")
# seems way too high...

### CONSTANTS

In [2]:
# CONSTANTS treated as variables for simulation

# inputs to the Drake equation
drake_best = dict(RS = 2, FP = 1, NE = 0.4, FL = 1, FI = 1, FC = 0.1, L = 1000)

drake_range = dict(
    RS = [1, 2, 3],
    FP = [0.05, 0.2, 0.5, 1.0, 2.0],
    NE = [0.02, 0.1, 0.4, 1.0],
    FL = [0.01, 0.1, 1],
    FI = [0.01, 0.1, 1],
    FC = [0.01, 0.1, 1],
    L = [100, 1_000, 10_000, 100_000, 1_000_000]
)

# approx timeframe on Earth
timeframes_best = dict(  # assumes Earth is representative
    YEARS_PLANETS_TO_HABITABLE = 2_000_000_000, 
    YEARS_HABITABLE_TO_LIFE = 1_000_000_000, 
    YEARS_LIFE_TO_COMPLEX_LIFE = 1_000_000_000, 
    YEARS_COMPLEX_TO_INTELLIGENCE = 200_000_000, 
    YEARS_INTELLIGENCE_TO_CULTURE = 20_000_000, 
    YEARS_CULTURE_TO_TECH = 200_000, 
    EXTINCTION_SIMPLE = 5_000_000_000, 
    EXTINCTION_COMPLEX = 2_000_000_000, 
    EXTINCTION_INTELLIGENT = 200_000_000, 
    EXTINCTION_CULTURAL = 1_000_000, 
    WEIBULL_SHAPE_PARAMETER = 0.4,  # k
    WEIBULL_SCALE_PARAMETER = 20_000  # lambda
)

timeframes_range = dict(  # assumes Earth is representative
    YEARS_PLANETS_TO_HABITABLE = [1_000_000_000, 2_000_000_000, 3_000_000_000], 
    YEARS_HABITABLE_TO_LIFE = [100_000_000, 1_000_000_000, 5_000_000_000], 
    YEARS_LIFE_TO_COMPLEX_LIFE = [100_000_000, 1_000_000_000, 5_000_000_000, 10_000_000_000], 
    YEARS_COMPLEX_TO_INTELLIGENCE = [100_000, 1_000_000, 50_000_000, 200_000_000, 1_000_000_000], 
    YEARS_INTELLIGENCE_TO_CULTURE = [10_000, 100_000, 2_000_000, 20_000_000, 200_000_000], 
    YEARS_CULTURE_TO_TECH = [1_000, 10_000, 100_000, 200_000, 2_000_000], 
    EXTINCTION_SIMPLE = [200_000_000, 1_000_000_000, 2_000_000_000, 5_000_000_000],  # these are mass extinction events
    EXTINCTION_COMPLEX = [200_000_000, 1_000_000_000, 2_000_000_000], # these are mass extinction events
    EXTINCTION_INTELLIGENT = [10_000, 100_000, 1_000_000, 10_000_000, 200_000_000, 1_000_000_000], 
    EXTINCTION_CULTURAL = [1_000, 10_000, 100_000, 1_000_000, 10_000_000], 
    WEIBULL_SHAPE_PARAMETER = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # k
    WEIBULL_SCALE_PARAMETER = [100, 250, 500, 750, 1000, 5_000, 10_000, 20_000, 50_000, 100_000]  # lambda
)

# POTANTIAL EXTINCTION CONSTANTS, HIGH UNCERTAINTY
# ## less optimistic - uncomment to use this extinction rate ##
# # prob_L(100), prob_L(1_000), prob_L(100_000)
# # (0.27111, 0.63212, 0.99995)
# WEIBULL_SHAPE_PARAMETER = 0.5  # k
# WEIBULL_SCALE_PARAMETER = 1000  # lambda / 63rd percentile
# ## best guess - uncomment to use this extinction rate ##
# # prob_L(100), prob_L(1_000), prob_L(100_000)
# # (0.11318, 0.26045, 0.85098)
# WEIBULL_SHAPE_PARAMETER = 0.4  # k
# WEIBULL_SCALE_PARAMETER = 20_000  # lambda / 63rd percentile


# STILL CONSTANTS

# galactic attributes
NUM_GALAXY = 100_000_000_000  # number of stars in a galaxy
GALAXY_RADIUS_LYR = 50_000  # radius of the galaxy in lightyears
MODERN_ERA = 10_000_000_000  # years since 2nd gen stars, order of magnitude approx

# time step increments for calculations
YEAR_STEPS = 1_000_000

In [3]:
#TODO: combine with above constants, integrate dicts

# best estimate for a solution to the Drake equation
RS = 2  # star formation per galaxy (ie, milky way)
FP = 1
NE = 0.4
FL = 1  # seems optimistic to me
FI = 1  # seems optimistic to me
FC = 0.1  # looks like a complete guess 
L = 1000  # maybe between 1000 and 100_000_000

NUM_GALAXY = 100_000_000_000  # number of stars in a galaxy
GALAXY_RADIUS_LYR = 50_000  # radius of the galaxy in lightyears
MODERN_ERA = 10_000_000_000  # years since 2nd gen stars, order of magnitude approx

# approx timeframe on Earth - assume Earth is representative
YEARS_PLANETS_TO_HABITABLE = 2_000_000_000
YEARS_HABITABLE_TO_LIFE = 1_000_000_000  # single cell sludge
YEARS_LIFE_TO_COMPLEX_LIFE = 1_000_000_000  # fish, birds, plants
YEARS_COMPLEX_TO_INTELLIGENCE = 200_000_000  # apes, dolphins
YEARS_INTELLIGENCE_TO_CULTURE = 20_000_000  # neandrethal, VERY HIGH UNCERTAINTY
YEARS_CULTURE_TO_TECH = 200_000  # humans, HIGH UNCERTAINTY

# estimates for mass extinctions in average years - these are very approximate
EXTINCTION_SIMPLE = 5_000_000_000  # approx limit to age of life on earth
EXTINCTION_COMPLEX = 2_000_000_000  # slightly more likely to go extinct
EXTINCTION_INTELLIGENT = 200_000_000  # guess: approx age of the dinosaurs
EXTINCTION_CULTURAL = 1_000_000  # guess: VERY HIGH UNCERTAINTY
# EXTINCTION_TECHNOLOGICAL = ??? # Weibull distribution is used to calculate this value

# time step increments for calculations
YEAR_STEPS = 1_000_000

# POTANTIAL EXTINCTION CONSTANTS, HIGH UNCERTAINTY
# ## less optimistic - uncomment to use this extinction rate ##
# # prob_L(100), prob_L(1_000), prob_L(100_000)
# # (0.27111, 0.63212, 0.99995)
# WEIBULL_SHAPE_PARAMETER = 0.5  # k
# WEIBULL_SCALE_PARAMETER = 1000  # lambda / 63rd percentile
## best guess - uncomment to use this extinction rate ##
# prob_L(100), prob_L(1_000), prob_L(100_000)
# (0.11318, 0.26045, 0.85098)
WEIBULL_SHAPE_PARAMETER = 0.4  # k
WEIBULL_SCALE_PARAMETER = 20_000  # lambda / 63rd percentile

### The Time-Dependent Drake Equation

In [4]:
%%time
history_of_life = TimeDependentDrake(output_year=10_000_000_000, P_tech_dominance=0.9)

history_of_life.tail()

NameError: name 'YEAR_STEPS' is not defined

In [ ]:
# plot of technological species over time

plt.figure(figsize=(12, 6))
plot = sns.lineplot(x=history_of_life.index/1_000_000_000, y=history_of_life.N);
plt.yticks(rotation=0);
plot.set_ylabel('Number of Technological Species');
plot.set_xlabel('Billions of Years');

# How to interpret these averages/expectations given 1M yr time steps

These numbers are based on the active instances over 1M year time steps. This isn't significant for values with slower
extinction values, but with N, these predictions are misleading. E.g., 100 technological species seems like
a lot, but if that's over 1M years, and they each survive 1k yr, there is less than 1 on average at the same time.

We need to convert these numbers into probabilities / histograms to gain insight into how likely we are to find life. 

In [ ]:
# number of technological lifeforms in 1M yr timestep at end of Drake calculation
N_1M_yr = int(round(history_of_life.iloc[[-1]].N))
N_1M_yr

In [ ]:
%%time

active_each_year = breakdown_by_year(N_1M_yr)

In [ ]:
percentage_table(active_each_year)

In [ ]:
plot_histogram(active_each_year, discrete=True)

In [ ]:
%%time
# hypothetical large number of technological lifeforms active in 1M yr timestep

# TESTING
N_large = 1_000
# histogram starts getting messy at 1k+, can't bin it easily because it's actually a barplot

active_each_year_large = breakdown_by_year(N_large)

In [ ]:
plot_histogram(active_each_year_large)

# implications of tech_dominance

In [ ]:
%%time
tech_dominance_1 = TimeDependentDrake(output_year=10_000_000_000, P_tech_dominance=1)

tech_dominance_1.tail()

In [ ]:
plt.figure(figsize=(12, 6))
plot = sns.lineplot(x=tech_dominance_1.index/1_000_000_000, y=tech_dominance_1.N);
plt.yticks(rotation=0);
plot.set_ylabel('Number of Technological Species');
plot.set_xlabel('Billions of Years');

In [ ]:
%%time
tech_dominance_0 = TimeDependentDrake(output_year=10_000_000_000, P_tech_dominance=0)

tech_dominance_0.tail()

In [ ]:
plt.figure(figsize=(12, 6))
plot = sns.lineplot(x=tech_dominance_0.index/1_000_000_000, y=tech_dominance_0.N);
plt.yticks(rotation=0);
plot.set_ylabel('Number of Technological Species');
plot.set_xlabel('Billions of Years');

# implications of longer time values (eg, 1T yr, steady state, etc)

In [ ]:
%%time
#TODO: haven't tested this as a function yet...
#TODO: write a loop to save csv in 10B yr increments
# combine csv
    # eg, history_of_life.to_csv('history_of_life_1T_yr.csv')  # or nested path
# plot it

epoch_steps = 1_000_000_000
final_year = 100_000_000_000
P_tech_dominance = 0.9

future_of_life = long_time(epoch_steps, final_year, P_tech_dominance = 0.9)

In [ ]:
# for 10B yr: 12s 
# for 100B yr: 124s
# for 1T yr: estimate 1240s = 21min
future_of_life.tail()

# steady state values
    what are steady state values?
    when will we reach approx steady state?
    can we match the time-invariant Drake Equation?

In [ ]:
%%time
# couldn't think of an algorithm that wouldn't take forever, so I brute-forced and grit
# should come up with an automated way to check different settings
P_tech_dominance = 0.9
steady_state = pd.DataFrame(
    [[0, 100_000_000_000, 40_000_000_000, 40_000_000_000, 974_975_916, 23_619_997, 6_551_317, 8_796, 6_262, 0]], 
    columns = columns,
    index=[0]
).set_index(index)

for i in range(2):
    temp_df = TimeDependentDrake(10_000_000_000*(i+1), P_tech_dominance, df_input=steady_state)
    steady_state = steady_state.append(temp_df.iloc[[-1]])
    
# print(steady_state.iloc[-1]-steady_state.iloc[0])
# print()
steady_state.tail()

In [ ]:
plot_histogram(breakdown_by_year(6_262), binwidth=5)

### Are these calculations equivalent to the orignial Drake Equation?

In [ ]:
# Using our best estimates, the Drake Equation predicts...
RS = 2
FP = 1
NE = 0.4
FL = 1
FI = 1
FC = 0.1
L = 5000  # to lead to approx the same species as above

Drake(RS, FP, NE, FL, FI, FC, L)
# this is within the range of estimates used in the prob_L() function, and 1k < L < 1M from current guesses

In [ ]:
int(weibull_mean())

In [ ]:
int(weibull_median())

mode is 0, because k < 0

Results not the same for the same L, but could be equivalent
    
    our mean > mode > original L
        there are a very large number of short-lived species
    let's check with a quick simulation
    we could also rerun using L constant or poisson to see if it gives the same results

In [ ]:
# simulation of prob_L

temp_list = []
temp_steps = 1_000

for i in range(1_000):
    temp_list.append([temp_steps * i, 1 - prob_L(temp_steps * i)])

df = pd.DataFrame(temp_list, columns=['Years', 'Prob_Survival'])

plt.figure(figsize=(12, 6))
plot = sns.lineplot(x=df['Years'], y=df['Prob_Survival']);
# plot.set(xscale='log');
plot.set_ylabel('Probability of Survival');
plt.ticklabel_format(style='plain', axis='x')
# plot.set_xlabel('Years');

In [ ]:
pd.options.display.float_format = '{:_.3f}'.format
df.head(10)
# 50% of species are extinct at 8k years

In [ ]:
# simulation of prob_L

temp_list = []
temp_steps = 10

for i in range(1_000):
    temp_list.append([temp_steps * i, 1 - prob_L(temp_steps * i)])

df = pd.DataFrame(temp_list, columns=['Years', 'Prob_Survival'])

plt.figure(figsize=(12, 6))
plot = sns.lineplot(x=df['Years'], y=df['Prob_Survival']);
# plot.set(xscale='log');
plot.set_ylabel('Probability of Survival');
plt.ticklabel_format(style='plain', axis='x')
# plot.set_xlabel('Years');

In [ ]:
df.head(10)
# 10% of species are extinct at 70 years

In [ ]:
# reset pandas format
pd.set_option('display.float_format', '{:_.0f}'.format)

# distance to closest species
#TODO: could make a formula to calculate the odds that there is x within y

In [ ]:
# closest living technological life
# most likely we're alone in the galaxy, but there could be 1 or 2, so
how_far(1)

In [ ]:
# likely max 2 other species
how_far(2)

In [ ]:
# steady state solution ~400 tech species
how_far(400)

##### Control Problem?

In [ ]:
# closest extinct technological species
# current year, with P_tech_dominance 0 or 1
how_far(10_000), how_far(40_000)

In [ ]:
# how many stars within 700 lyr?
stars_within(724.6)
# we'd probably notice 2.5M stars being converted into paperclips
# we should at least be able to detect a runaway AI

In [ ]:
ai_propagation_speed_low = 0.0001  # percentage of the speed of light that superintelligent AI can travel
ai_propagation_speed_high = 0.01  # percentage of the speed of light that superintelligent AI can travel
chance_of_control_problem_low = 0.01  # proportion of extinctions due to runaway paperclip optimiser
chance_of_control_problem_high = 0.1  # proportion of extinctions due to runaway stamp collector

# years for rogue AI to travel here
pd.DataFrame(
    [[how_far(11_000 * chance_of_control_problem_low) / ai_propagation_speed_low, 
      how_far(11_000 * chance_of_control_problem_high) / ai_propagation_speed_high
    ]], columns=['Years Until Invasion - High', 'Years Until Invasion - Low']
)

In [ ]:
# plot of extinct technological species over time

plt.figure(figsize=(12, 6))
plot = sns.lineplot(x=history_of_life.index/1_000_000, y=history_of_life.N_extinct);

plot.set(xlim=(9_900, 10_000),ylim=(10_800, 11_500));
plt.yticks(rotation=0);
plot.set_ylabel('Number of Extinct Technological Species');
plot.set_xlabel('Millions of Years');

In [ ]:
# kind of seems like we should have been converted into paperclips by now...
# i feel like we'll notice before 10k lyr**2 of stars were converted to paperclips, but you never know

In [ ]:
# how close are the closest paperclip monsters
how_far(11_000 * chance_of_control_problem_low), how_far(11_000 * chance_of_control_problem_high)

In [ ]:
# how many stars within 700 lyr?
stars_within(2664.0)
# we'd very likely notice 90M stars being converted into stamps



**we should have plenty of notice**

    unless we mistake rogue AI for very common astrophysical phemomena 

# Monte Carlo Simulation

In [ ]:
# test randomisation
randomise_constants(drake_range, timeframes_range)
df1 = pd.DataFrame([RS, FP, NE, FL, FI, FC, L, YEARS_PLANETS_TO_HABITABLE, YEARS_HABITABLE_TO_LIFE, YEARS_LIFE_TO_COMPLEX_LIFE, YEARS_COMPLEX_TO_INTELLIGENCE, YEARS_INTELLIGENCE_TO_CULTURE, YEARS_CULTURE_TO_TECH, EXTINCTION_SIMPLE, EXTINCTION_COMPLEX, EXTINCTION_INTELLIGENT, EXTINCTION_CULTURAL, WEIBULL_SHAPE_PARAMETER, WEIBULL_SCALE_PARAMETER])
reset_default_constants(drake_best, timeframes_best)
df2 = pd.DataFrame([RS, FP, NE, FL, FI, FC, L, YEARS_PLANETS_TO_HABITABLE, YEARS_HABITABLE_TO_LIFE, YEARS_LIFE_TO_COMPLEX_LIFE, YEARS_COMPLEX_TO_INTELLIGENCE, YEARS_INTELLIGENCE_TO_CULTURE, YEARS_CULTURE_TO_TECH, EXTINCTION_SIMPLE, EXTINCTION_COMPLEX, EXTINCTION_INTELLIGENT, EXTINCTION_CULTURAL, WEIBULL_SHAPE_PARAMETER, WEIBULL_SCALE_PARAMETER])
pd.concat([df1, df2], axis=1)

### Simulations

##### Original Drake Equation

In [ ]:
%%time

simulation = []

for i in range(1_000_000):
    randomise_constants(drake_range, timeframes_range)
    simulation.append(Drake(RS, FP, NE, FL, FI, FC, L))

reset_default_constants(drake_best, timeframes_best)

pd.DataFrame(simulation).describe()

In [ ]:
# confirm that defaults are reset
Drake(RS, FP, NE, FL, FI, FC, L)  # defaults lead to 80

##### Time Dependent Drake

In [ ]:
%%time

output_year = 1_000_000_000
simulation2 = []

for i in range(10):
    randomise_constants(drake_range, timeframes_range)
    N_temp = TimeDependentDrake(output_year=output_year, P_tech_dominance=np.random.random())['N']
    simulation2.append(N_temp.iloc[-1])

reset_default_constants(drake_best, timeframes_best)

pd.DataFrame(simulation2)
# pd.DataFrame(simulation2).describe()

In [ ]:
pd.set_option('display.float_format', '{:_.0f}'.format)

In [ ]:
%%time

output_year = 1_000_000_000
simulation2 = pd.DataFrame()

for i in range(100):
    randomise_constants(drake_range, timeframes_range)
    df_temp = TimeDependentDrake(output_year=output_year, P_tech_dominance=np.random.random())
    simulation2 = simulation2.append(df_temp.iloc[-1])

reset_default_constants(drake_best, timeframes_best)


In [ ]:
simulation2.describe()

In [ ]:
pd.set_option('display.max_rows', 100)
simulation2

1M years, 100 trials -> 2min 38s
trials should be linear, years may not be.
lets check... looks linear

so 
10M years, 100 trials -> 26min
10M years, 1000 trials -> 4hr 23min
10M years, 1M trials -> 6 months

I have to cut the lawn and play Ghost of Tsushima, let's do the 4.5 hour simulation!

In [ ]:
%%time
temp = TimeDependentDrake(output_year=100_000_000, P_tech_dominance=0.9)

In [ ]:
%%time
temp = TimeDependentDrake(output_year=1_000_000_000, P_tech_dominance=0.9)

In [ ]:
%%time
temp = TimeDependentDrake(output_year=10_000_000_000, P_tech_dominance=0.9)

In [ ]:
%%time

# TEST: Make sure it works before wasting 5 hours

output_year = 1_000_000_000
n_sims = 10
simulation2 = pd.DataFrame()

for i in range(n_sims):
    randomise_constants(drake_range, timeframes_range)
    df_temp = TimeDependentDrake(output_year=output_year, P_tech_dominance=np.random.random())
    df_param_temp = pd.DataFrame([[
        RS, FP, NE, FL, FI, FC, L, YEARS_PLANETS_TO_HABITABLE, YEARS_HABITABLE_TO_LIFE, 
        YEARS_LIFE_TO_COMPLEX_LIFE, YEARS_COMPLEX_TO_INTELLIGENCE, YEARS_INTELLIGENCE_TO_CULTURE, 
        YEARS_CULTURE_TO_TECH, EXTINCTION_SIMPLE, EXTINCTION_COMPLEX, EXTINCTION_INTELLIGENT, 
        EXTINCTION_CULTURAL, WEIBULL_SHAPE_PARAMETER, WEIBULL_SCALE_PARAMETER
        ]],
        columns = [
        'RS', 'FP', 'NE', 'FL', 'FI', 'FC', 'L', 'YEARS_PLANETS_TO_HABITABLE', 'YEARS_HABITABLE_TO_LIFE',
        'YEARS_LIFE_TO_COMPLEX_LIFE', 'YEARS_COMPLEX_TO_INTELLIGENCE', 'YEARS_INTELLIGENCE_TO_CULTURE',
        'YEARS_CULTURE_TO_TECH', 'EXTINCTION_SIMPLE', 'EXTINCTION_COMPLEX', 'EXTINCTION_INTELLIGENT',
        'EXTINCTION_CULTURAL', 'WEIBULL_SHAPE_PARAMETER', 'WEIBULL_SCALE_PARAMETER'
        ] 
    )
    simulation2 = simulation2.append(df_temp.iloc[[-1]].reset_index().join(df_param_temp))

reset_default_constants(drake_best, timeframes_best)
simulation2.to_csv('time_dep_monte_carlo_10Myr_1000trials.csv', index=False)

In [ ]:
%%time

# SIMULATION: 10M yr,  1k trials, should take 5 hours to run

output_year = 10_000_000_000
n_sims = 1_000
simulation2 = pd.DataFrame()

for i in range(n_sims):
    randomise_constants(drake_range, timeframes_range)
    df_temp = TimeDependentDrake(output_year=output_year, P_tech_dominance=np.random.random())
    df_param_temp = pd.DataFrame([[
        RS, FP, NE, FL, FI, FC, L, YEARS_PLANETS_TO_HABITABLE, YEARS_HABITABLE_TO_LIFE, 
        YEARS_LIFE_TO_COMPLEX_LIFE, YEARS_COMPLEX_TO_INTELLIGENCE, YEARS_INTELLIGENCE_TO_CULTURE, 
        YEARS_CULTURE_TO_TECH, EXTINCTION_SIMPLE, EXTINCTION_COMPLEX, EXTINCTION_INTELLIGENT, 
        EXTINCTION_CULTURAL, WEIBULL_SHAPE_PARAMETER, WEIBULL_SCALE_PARAMETER
        ]],
        columns = [
        'RS', 'FP', 'NE', 'FL', 'FI', 'FC', 'L', 'YEARS_PLANETS_TO_HABITABLE', 'YEARS_HABITABLE_TO_LIFE',
        'YEARS_LIFE_TO_COMPLEX_LIFE', 'YEARS_COMPLEX_TO_INTELLIGENCE', 'YEARS_INTELLIGENCE_TO_CULTURE',
        'YEARS_CULTURE_TO_TECH', 'EXTINCTION_SIMPLE', 'EXTINCTION_COMPLEX', 'EXTINCTION_INTELLIGENT',
        'EXTINCTION_CULTURAL', 'WEIBULL_SHAPE_PARAMETER', 'WEIBULL_SCALE_PARAMETER'
        ] 
    )
    simulation2 = simulation2.append(df_temp.iloc[[-1]].reset_index().join(df_param_temp))

reset_default_constants(drake_best, timeframes_best)

now = datetime.datetime.now().strftime("%m%d_%H%M")
simulation2.to_csv(f'time_dep_monte_carlo_10Myr_1000trials_{now}.csv', index=False)

In [ ]:
simulation2.N.describe()

In [ ]:
"""
#  TODO: 
    set up a better loop, output multiple smaller csv
    create a function to import and combine csv into df
    look at results and figure out if any of the ranges are 0s or huge
        use these as flags
        monte carlo moderate values
    figure out which parameters / functions are leading to negative numbers
        maybe rounding - revise
    investigate 10M or 100M timesteps to speed up the simulation process
    make a library of functions to import, and an abbreviated workbook with only conclusions
"""